# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,revda,56.8010,59.9303,1.68,86,100,5.47,RU,1713900853
1,1,afaahiti,-17.7500,-149.2833,28.92,78,1,4.72,PF,1713901122
2,2,grytviken,-54.2811,-36.5092,-2.25,78,41,14.06,GS,1713901122
3,3,itoman,26.1247,127.6694,23.21,96,75,2.57,JP,1713901122
4,4,lebu,-37.6167,-73.6500,13.46,57,82,2.35,CL,1713901122


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
#%%capture --no-display

# Configure the map plot
humid_city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM",
                                      scale = 1
                                     )

# Display the map
humid_city_map



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# # Narrow down cities that fit criteria and drop any results with null values

ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 25)
                                       & (city_data_df["Max Temp"] > 20) 
                                       & (city_data_df["Cloudiness"] < 20)
                                       & (city_data_df["Wind Speed"] < 5)
                                      ]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,ribeira brava,32.6500,-17.0667,21.51,74,8,0.74,PT,1713901129
31,31,tsiombe,-25.3000,45.4833,21.15,78,7,3.55,MG,1713901131
32,32,albany,42.6001,-73.9662,20.69,27,12,3.13,US,1713900940
52,52,buka,40.8108,69.1986,21.13,38,17,2.94,UZ,1713901133
71,71,shakawe,-18.3667,21.8500,20.99,24,0,2.27,BW,1713901135
134,134,taunggyi,20.7833,97.0333,20.95,62,0,1.59,MM,1713901138
136,136,greenville,35.6127,-77.3663,21.18,34,0,4.12,US,1713900938
230,230,santa terezinha,-29.9002,-50.0904,24.83,79,3,0.78,BR,1713901158
232,232,iquique,-20.2208,-70.1431,23.02,77,1,4.31,CL,1713900882
248,248,raton,36.9034,-104.4391,22.34,20,0,3.09,US,1713901159


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,ribeira brava,PT,32.6500,-17.0667,74,
31,tsiombe,MG,-25.3000,45.4833,78,
32,albany,US,42.6001,-73.9662,27,
52,buka,UZ,40.8108,69.1986,38,
71,shakawe,BW,-18.3667,21.8500,24,
134,taunggyi,MM,20.7833,97.0333,62,
136,greenville,US,35.6127,-77.3663,34,
230,santa terezinha,BR,-29.9002,-50.0904,79,
232,iquique,CL,-20.2208,-70.1431,77,
248,raton,US,36.9034,-104.4391,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories": categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city = row["City"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
tsiombe - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
buka - nearest hotel: No hotel found
shakawe - nearest hotel: EdenWood guesthouse
taunggyi - nearest hotel: Golden Island Cottages
greenville - nearest hotel: Quality Inn
santa terezinha - nearest hotel: Pousada Santa Terezinha
iquique - nearest hotel: Hostal Vivar
raton - nearest hotel: Travelodge
hwange - nearest hotel: No hotel found
maputo - nearest hotel: Mozaika
aral - nearest hotel: No hotel found
riverton - nearest hotel: Homewood Suites
gwanda - nearest hotel: No hotel found
kaiyuan - nearest hotel: No hotel found
rundu - nearest hotel: Kavango River Lodge
marshall - nearest hotel: Wyndham Garden Lake-Guntersville
behompy - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
25,ribeira brava,PT,32.6500,-17.0667,74,Cheerfulway Bravamar Hotel
31,tsiombe,MG,-25.3000,45.4833,78,No hotel found
32,albany,US,42.6001,-73.9662,27,No hotel found
52,buka,UZ,40.8108,69.1986,38,No hotel found
71,shakawe,BW,-18.3667,21.8500,24,EdenWood guesthouse
134,taunggyi,MM,20.7833,97.0333,62,Golden Island Cottages
136,greenville,US,35.6127,-77.3663,34,Quality Inn
230,santa terezinha,BR,-29.9002,-50.0904,79,Pousada Santa Terezinha
232,iquique,CL,-20.2208,-70.1431,77,Hostal Vivar
248,raton,US,36.9034,-104.4391,20,Travelodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.6,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"],
                                   scale = 1
                                  )

# Display the map
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)